# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

#configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Reading from csv into dataframe using pandas
weatherdataframe = pd.read_csv("../WeatherPy/Weather CSV")
weatherdataframe.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Pevek,69.70,170.31,25.84,94,9,16.69,RU,1603085585
1,1,Albany,42.60,-73.97,52.00,70,66,4.83,US,1603085587
2,2,Minas,-34.38,-55.24,68.00,82,1,9.17,UY,1603085588
3,3,Puerto Ayora,-0.74,-90.35,71.01,89,99,8.01,EC,1603085056
4,4,Prince Rupert,54.32,-130.32,42.80,100,90,5.82,CA,1603085457


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Store latitude and longitude in locations:
locations = weatherdataframe[["Lat", "Lng"]]

#Store humidity values. Fill NaN values and convert to float.

humidity = weatherdataframe["Humidity"].astype(float)

#Calculating the maximum humidity for plotting:
humidity_max = humidity.max()
humidity_max

100.0

In [4]:
#Plot the heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = humidity_max, point_radius = 1)

#Add the layer to the map
fig.add_layer(heat_layer)

#Display figure
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrowed dataframe to find ideal weather condition: Temp
idealtemp_df = weatherdataframe.loc[(weatherdataframe["Max Temp"]<80) & (weatherdataframe["Max Temp"]>70)]

In [6]:
#Narrowed dataframe to find ideal weather condition: Wind Speed
idealwind_df = weatherdataframe.loc[weatherdataframe["Wind Speed"]<10]

In [7]:
#Narrowed dataframe to find ideal weather condition: Zero Cloudiness
idealwind_df = weatherdataframe.loc[weatherdataframe["Cloudiness"]==0]

In [8]:
#Combined all conditions into a single dataframe:

allconditionslong_df= weatherdataframe.loc[(weatherdataframe["Max Temp"]<80) & (weatherdataframe["Max Temp"]>70) & (weatherdataframe["Wind Speed"]<10) & (weatherdataframe["Cloudiness"]==0)]

#Setting limit of 10 results
allconditions_df=allconditionslong_df.head(10)

allconditions_df





,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,Agadez,19.75,10.25,75.94,18,0,2.91,NE,1603085597
44,44,Tambacounda,13.75,-13.76,76.17,64,0,3.24,SN,1603085633
159,159,Lubango,-14.92,13.49,70.90,35,0,1.12,AO,1603085759
170,170,Doka,13.52,35.77,79.47,62,0,8.16,SD,1603085771
177,177,Frontera,26.93,-101.45,74.93,55,0,3.24,MX,1603085779
182,182,Kiryat Gat,31.61,34.76,73.99,81,0,3.67,IL,1603085784
203,203,Lichinga,-13.31,35.24,70.20,56,0,9.57,MZ,1603085808
244,244,Changli,39.70,119.15,72.45,22,0,9.82,CN,1603085852
331,331,Numan,9.47,12.03,76.33,80,0,8.01,NG,1603085946
357,357,Ondjiva,-17.07,15.73,74.95,19,0,4.45,AO,1603085974


In [9]:
allconditions_df.count()

Unnamed: 0    10
City          10
Lat           10
Lng           10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
Date          10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#New hotel dataframe
hotel_df = allconditions_df[["City","Lat","Lng","Country"]].copy()

In [11]:
#Creating new hotel name column for dataframe
hotel_df["Hotel Name"]= ""
hotel_df

,City,Lat,Lng,Country,Hotel Name
11,Agadez,19.75,10.25,NE,
44,Tambacounda,13.75,-13.76,SN,
159,Lubango,-14.92,13.49,AO,
170,Doka,13.52,35.77,SD,
177,Frontera,26.93,-101.45,MX,
182,Kiryat Gat,31.61,34.76,IL,
203,Lichinga,-13.31,35.24,MZ,
244,Changli,39.70,119.15,CN,
331,Numan,9.47,12.03,NG,
357,Ondjiva,-17.07,15.73,AO,


In [12]:
#base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#creating parameters dictionary    
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # Get coordinate data type from df
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    #Get name data from df
    city = row["City"]

    # Add location to params dictionary (necessary for nearby)
    params["location"] = f"{latitude},{longitude}"
    
    # Assemble url, make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Nearby hotel in {city}: {results[0]['name']}.")
        
        #Putting this iterated info into a dataframe
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    #Break
    time.sleep(1)


Retrieving Results for Index 11: Agadez.
Missing field/result... skipping.
Retrieving Results for Index 44: Tambacounda.
Missing field/result... skipping.
Retrieving Results for Index 159: Lubango.
Nearby hotel in Lubango: Resort Pululukua.
Retrieving Results for Index 170: Doka.
Missing field/result... skipping.
Retrieving Results for Index 177: Frontera.
Nearby hotel in Frontera: Holiday Inn Monclova.
Retrieving Results for Index 182: Kiryat Gat.
Nearby hotel in Kiryat Gat: Desert Gat.
Retrieving Results for Index 203: Lichinga.
Nearby hotel in Lichinga: Girassol Lichinga.
Retrieving Results for Index 244: Changli.
Nearby hotel in Changli: Hesheng Hotel.
Retrieving Results for Index 331: Numan.
Nearby hotel in Numan: Numan Hotel.
Retrieving Results for Index 357: Ondjiva.
Nearby hotel in Ondjiva: Villa Okapale.


In [13]:
#Checking if hotels printed
hotel_df

,City,Lat,Lng,Country,Hotel Name
11,Agadez,19.75,10.25,NE,
44,Tambacounda,13.75,-13.76,SN,
159,Lubango,-14.92,13.49,AO,Resort Pululukua
170,Doka,13.52,35.77,SD,
177,Frontera,26.93,-101.45,MX,Holiday Inn Monclova
182,Kiryat Gat,31.61,34.76,IL,Desert Gat
203,Lichinga,-13.31,35.24,MZ,Girassol Lichinga
244,Changli,39.70,119.15,CN,Hesheng Hotel
331,Numan,9.47,12.03,NG,Numan Hotel
357,Ondjiva,-17.07,15.73,AO,Villa Okapale


In [14]:
#Creating hotel_info dataframe
#hotel_info=[["City","Country","Hotel Name"]]

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map, use hotel_info to fill the info box
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))